# LangGraph Agent

This notebook demonstrates how to build sophisticated RAG (Retrieval-Augmented Generation) systems using **LangGraph** - a powerful framework for building stateful, multi-actor applications with large language models.

## What is LangGraph?
LangGraph is a library for building stateful, multi-step applications with LLMs. It extends LangChain with graph-based workflows that can:
- **Maintain State**: Preserve context across multiple steps
- **Conditional Logic**: Make decisions based on intermediate results
- **Tool Integration**: Seamlessly incorporate external tools and APIs
- **Memory Management**: Handle conversation history and persistence

## LangGraph vs Traditional RAG
- **Traditional RAG**: Simple pipeline (Query → Retrieve → Generate)
- **LangGraph RAG**: Stateful workflow with conditional paths, tool calls, and memory

## What We'll Build
We'll create two different RAG systems:
1. **Custom LangGraph RAG**: A manually constructed graph with explicit state management
2. **ReAct Agent**: A pre-built agent that uses Reasoning and Acting patterns

## Key Concepts Covered
1. **MessagesState**: LangGraph's built-in state for managing conversation history
2. **Tools**: Functions the LLM can call to retrieve information
3. **Conditional Edges**: Dynamic routing based on LLM decisions
4. **Memory/Checkpointing**: Conversation persistence across sessions
5. **Graph Visualization**: Understanding workflow structure
6. **Multi-turn Conversations**: Handling follow-up questions with context

## Step 1: Environment Setup and API Key Management

This section handles the secure loading of API credentials for Azure OpenAI.

### Your Task:
Set up environment variables for Azure OpenAI API access.

**Steps:**
1. Import required modules:
   - `load_dotenv` from `dotenv` (loads environment variables from .env file)
   - `getpass` (for secure password input)
   - `os` (for environment variable access)
2. Call `load_dotenv()` to load environment variables from a `.env` file if present
3. Check if `AZURE_OPENAI_API_KEY` exists in environment variables
4. If not found, prompt the user securely: `os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter your Azure OpenAI API key: ")`

**Security Best Practice:** 
- Never hardcode API keys in code
- Use environment variables or secure prompts
- Works both locally (with `.env` file) and in production (with system environment variables)

**Expected Output:** No visible output, but the API key will be securely stored

In [ ]:
# TODO: Import load_dotenv from dotenv, getpass, and os


# TODO: Load environment variables from .env file


# TODO: Check if AZURE_OPENAI_API_KEY exists, if not, prompt for it

## Step 2: Initialize the Embedding Model

**Embeddings** are the foundation of semantic search in RAG systems. They convert text into numerical vectors that capture meaning.

### Your Task:
Configure Azure OpenAI embeddings for converting text to vectors.

**Steps:**
1. Import `AzureOpenAIEmbeddings` from `langchain_openai`
2. Create an embeddings instance with these parameters:
   - `azure_endpoint="https://aoi-ext-eus-aiml-profx-01.openai.azure.com/"`
   - `api_key=os.environ["AZURE_OPENAI_API_KEY"]`
   - `model="text-embedding-ada-002"`
   - `api_version="2024-12-01-preview"`

### How Embeddings Work:
- **Input**: Text strings (documents, queries)
- **Output**: High-dimensional vectors (1536 dimensions for `text-embedding-ada-002`)
- **Property**: Semantically similar text produces similar vectors

**Expected Output:** No output, but the `embeddings` object will be ready to use

In [ ]:
# TODO: Import AzureOpenAIEmbeddings from langchain_openai


# TODO: Create an AzureOpenAIEmbeddings instance with the required parameters

## Step 3: Initialize the Language Model (LLM)

**Large Language Models (LLMs)** generate human-like text responses based on input prompts.

### Your Task:
Set up Azure Chat OpenAI for generating answers.

**Steps:**
1. Import `AzureChatOpenAI` from `langchain_openai`
2. Create an LLM instance with these parameters:
   - `azure_endpoint="https://aoi-ext-eus-aiml-profx-01.openai.azure.com/"`
   - `api_key=os.environ["AZURE_OPENAI_API_KEY"]`
   - `model="gpt-4o"`
   - `api_version="2024-12-01-preview"`

### Role in LangGraph:
The LLM will:
1. Process user queries
2. Decide which tools to call
3. Generate responses based on retrieved context
4. Maintain conversation flow

**Expected Output:** No output, but the `llm` object will be ready to generate responses

In [ ]:
# TODO: Import AzureChatOpenAI from langchain_openai


# TODO: Create an AzureChatOpenAI instance with the required parameters

## Step 4: Load and Prepare Document Data

**Document Loading** is crucial for creating a knowledge base for our RAG system.

### Your Task:
Load documents from a web source and prepare them for vector storage.

**Steps:**
1. Import required classes:
   - `WebBaseLoader` from `langchain_community.document_loaders`
   - `RecursiveCharacterTextSplitter` from `langchain_text_splitters`
2. Create a loader: `loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")`
3. Load documents: `docs = loader.load()`
4. Create a text splitter with `chunk_size=1000` and `chunk_overlap=200`
5. Split the documents: `all_splits = text_splitter.split_documents(docs)`

### Why Document Splitting?
- Large documents exceed embedding model limits
- Smaller chunks improve retrieval accuracy
- Overlap ensures context isn't lost at chunk boundaries

**Expected Output:** Information about the number of document chunks created

In [ ]:
# TODO: Import WebBaseLoader and RecursiveCharacterTextSplitter


# TODO: Create a WebBaseLoader and load documents


# TODO: Create a text splitter and split the documents


# TODO: Print the number of document chunks

## Step 5: Create Vector Store with Document Embeddings

**Vector Stores** enable semantic search by storing document embeddings and providing similarity search capabilities.

### Your Task:
Create a vector store and populate it with document embeddings.

**Steps:**
1. Import `InMemoryVectorStore` from `langchain_core.vectorstores`
2. Create a vector store: `vector_store = InMemoryVectorStore(embeddings)`
3. Add documents to the store: `vector_store.add_documents(all_splits)`

### How Vector Stores Work:
1. **Embedding Creation**: Each document chunk is converted to a vector using the embedding model
2. **Storage**: Vectors are stored with their associated text and metadata
3. **Retrieval**: Query vectors are compared against stored vectors to find similar content

### InMemoryVectorStore Benefits:
- Fast for development and small datasets
- No external dependencies
- Perfect for learning and prototyping

**Expected Output:** Confirmation that documents have been indexed

In [ ]:
# TODO: Import InMemoryVectorStore from langchain_core.vectorstores


# TODO: Create an InMemoryVectorStore instance


# TODO: Add documents to the vector store


# TODO: Print confirmation message

## Step 6: Create the Retrieval Tool

**Tools** in LangGraph are functions that the LLM can call to perform specific tasks. We'll create a retrieval tool that searches our vector store.

### Your Task:
Create a retrieval tool that the LLM can use to search for relevant documents.

**Steps:**
1. Define a retrieval function that:
   - Takes a query string as input
   - Uses `vector_store.similarity_search(query, k=3)` to find relevant documents
   - Returns the concatenated content of retrieved documents
2. Use the `@tool` decorator to convert the function into a LangChain tool
3. Test the tool with a sample query

### Tool Design Principles:
- **Clear Purpose**: Each tool should have a specific, well-defined function
- **Good Documentation**: Include docstrings that describe the tool's purpose and parameters
- **Error Handling**: Handle cases where no relevant documents are found

**Expected Output:** A working retrieval tool that can search the vector store

In [ ]:
# TODO: Import the tool decorator from langchain_core.tools


# TODO: Define a retrieval function with the @tool decorator
# The function should take a query parameter and return relevant documents


# TODO: Test the retrieval tool with a sample query

## Step 7: Set Up LangGraph State Management

**State Management** is what makes LangGraph powerful. It allows the system to maintain context across multiple steps.

### Your Task:
Import and understand LangGraph's MessagesState for managing conversation history.

**Steps:**
1. Import `MessagesState` from `langgraph.graph`
2. Import `SystemMessage` from `langchain_core.messages`

### What is MessagesState?
- **Built-in State**: Pre-defined state schema for chat applications
- **Message History**: Automatically tracks conversation messages
- **Persistence**: Can be saved and restored across sessions
- **Flexible**: Can be extended with custom fields

### Message Types:
- **SystemMessage**: Sets the AI's role and behavior
- **HumanMessage**: User inputs
- **AIMessage**: AI responses
- **ToolMessage**: Tool execution results

**Expected Output:** No output, but required imports will be ready

In [ ]:
# TODO: Import MessagesState from langgraph.graph


# TODO: Import SystemMessage from langchain_core.messages

## Step 8: Build the LangGraph Workflow

**LangGraph Workflows** define how the system processes information through a series of connected nodes.

### Your Task:
Create a LangGraph state graph and define the workflow nodes.

**Steps:**
1. Import required classes:
   - `StateGraph` and `END` from `langgraph.graph`
   - `ToolNode` from `langgraph.prebuilt`
2. Create a StateGraph: `graph_builder = StateGraph(MessagesState)`
3. Create a tools list containing your retrieval tool
4. Create a ToolNode: `tools = ToolNode([retrieve])`
5. Bind tools to the LLM: `llm = llm.bind_tools([retrieve])`

### Graph Components:
- **Nodes**: Individual processing steps (chatbot, tools)
- **Edges**: Connections between nodes
- **State**: Shared data that flows through the graph
- **Conditional Logic**: Dynamic routing based on conditions

**Expected Output:** A configured graph builder ready for node definition

In [ ]:
# TODO: Import StateGraph, END from langgraph.graph and ToolNode from langgraph.prebuilt


# TODO: Create a StateGraph with MessagesState


# TODO: Create a tools list and ToolNode


# TODO: Bind tools to the LLM

## Step 9: Define Graph Nodes and Logic

**Graph Nodes** are the processing units in our LangGraph workflow. Each node performs a specific function.

### Your Task:
Define the chatbot node and tool routing logic.

**Steps:**
1. Define a `chatbot` function that:
   - Takes the current state as input
   - Adds a system message if this is the first interaction
   - Invokes the LLM with the current messages
   - Returns the response

2. Define a `route_tools` function that:
   - Checks if the last AI message has tool calls
   - Returns "tools" if tools are called, otherwise END

3. Add nodes to the graph:
   - `graph_builder.add_node("chatbot", chatbot)`
   - `graph_builder.add_node("tools", tools)`

4. Add edges:
   - `graph_builder.add_conditional_edge("chatbot", route_tools, {"tools": "tools", END: END})`
   - `graph_builder.add_edge("tools", "chatbot")`
   - `graph_builder.set_entry_point("chatbot")`

**Expected Output:** A fully configured graph ready for compilation

In [ ]:
# TODO: Define the chatbot function
def chatbot(state: MessagesState):
    # TODO: Add system message if no messages exist
    
    # TODO: Invoke the LLM and return the response
    pass

# TODO: Define the route_tools function
def route_tools(state: MessagesState):
    # TODO: Check if the last message has tool calls
    # Return "tools" if tools are called, otherwise END
    pass

# TODO: Add nodes to the graph


# TODO: Add edges to connect the nodes


# TODO: Set the entry point

## Step 10: Compile and Test the Graph

**Graph Compilation** converts your graph definition into an executable workflow.

### Your Task:
Compile the graph and test it with a simple query.

**Steps:**
1. Compile the graph: `graph = graph_builder.compile()`
2. Create a test configuration dictionary
3. Test with a simple query using `graph.invoke()`
4. Print the response

### Graph Compilation Process:
- **Validation**: Checks that all nodes and edges are properly defined
- **Optimization**: Optimizes the execution path
- **Execution Ready**: Creates a runnable graph instance

**Expected Output:** A working LangGraph that can answer questions using RAG

In [ ]:
# TODO: Compile the graph


# TODO: Create a test configuration


# TODO: Test the graph with a simple query


# TODO: Print the response

## Step 11: Visualize the Graph Structure

**Graph Visualization** helps understand the workflow structure and debug issues.

### Your Task:
Generate a visual representation of your LangGraph workflow.

**Steps:**
1. Use `graph.get_graph().draw_mermaid_png()` to generate a diagram
2. Display the diagram using appropriate visualization methods

### Understanding the Diagram:
- **Nodes**: Rectangular boxes representing processing steps
- **Edges**: Arrows showing the flow between nodes
- **Conditional Edges**: Diamond shapes indicating decision points
- **Entry/Exit Points**: Special nodes marking start and end

**Expected Output:** A visual diagram of your LangGraph workflow

In [ ]:
# TODO: Generate and display the graph visualization

## Step 12: Test Complex Queries

**Complex Queries** test the system's ability to handle multi-step reasoning and tool usage.

### Your Task:
Test the system with a more complex question that requires retrieval.

**Expected Output:** A comprehensive answer based on retrieved documents

In [ ]:
# TODO: Test with a complex query about agents
input_message = "What are the key components of an LLM-powered agent according to the document?"

# TODO: Invoke the graph and print the response

## Step 13: Add Memory and Persistence

**Memory and Persistence** allow the system to maintain conversation context across multiple interactions.

### Your Task:
Add memory capabilities to enable multi-turn conversations.

**Steps:**
1. Import `MemorySaver` from `langgraph.checkpoint.memory`
2. Create a memory instance: `memory = MemorySaver()`
3. Recompile the graph with memory: `agent_executor = graph_builder.compile(checkpointer=memory)`
4. Create a config with a thread ID for conversation tracking

### Benefits of Memory:
- **Context Continuity**: Maintains conversation history
- **Follow-up Questions**: Can reference previous exchanges
- **Session Management**: Different threads for different conversations
- **Persistence**: Conversations can be resumed later

**Expected Output:** A memory-enabled agent ready for multi-turn conversations

In [ ]:
# TODO: Import MemorySaver from langgraph.checkpoint.memory


# TODO: Create a memory instance and recompile the graph


# TODO: Create a config with thread ID

## Step 14: Test Multi-turn Conversations

**Multi-turn Conversations** demonstrate the system's ability to maintain context across multiple exchanges.

### Your Task:
Test the memory-enabled system with follow-up questions.

**Steps:**
1. Ask an initial question
2. Ask a follow-up question that references the previous response
3. Observe how the system maintains context

**Expected Output:** Contextually aware responses that reference previous parts of the conversation

In [ ]:
# TODO: Test the first question
input_message = "What is retrieval-augmented generation?"

# TODO: Invoke the agent and print the response

## Step 15: Ask Follow-up Questions

### Your Task:
Ask a follow-up question to test conversation memory.

**Expected Output:** A response that references the previous conversation context

In [ ]:
# TODO: Ask a follow-up question
input_message_2 = "What are the advantages of this approach?"

# TODO: Invoke the agent and print the response

## Step 16: Build a ReAct Agent (Alternative Approach)

**ReAct Agents** use a pre-built pattern for Reasoning and Acting. They automatically decide when and how to use tools.

### Your Task:
Create a ReAct agent using LangGraph's built-in functionality.

**Steps:**
1. Import `create_react_agent` from `langgraph.prebuilt`
2. Create a ReAct agent with the LLM and tools list
3. Test the agent with the same queries

### ReAct vs Custom Graph:
- **ReAct**: Pre-built, standardized reasoning pattern
- **Custom**: Full control over workflow and logic
- **Use ReAct when**: You want a standard agent pattern
- **Use Custom when**: You need specific workflow logic

**Expected Output:** A working ReAct agent that performs similar functions to your custom graph

In [ ]:
# TODO: Import create_react_agent from langgraph.prebuilt


# TODO: Create a ReAct agent


# TODO: Test the ReAct agent

## Step 17: Compare Approaches

**Approach Comparison** helps understand when to use different LangGraph patterns.

### Your Task:
Test both the custom graph and ReAct agent with the same query and compare results.

### Key Differences:
- **Implementation Complexity**: Custom requires more code, ReAct is simpler
- **Flexibility**: Custom allows precise control, ReAct follows standard patterns
- **Debugging**: Custom easier to debug, ReAct more opaque
- **Performance**: Both should perform similarly for basic RAG tasks

**Expected Output:** Insights into which approach works better for different scenarios

In [ ]:
# TODO: Test both approaches with the same query and compare

## Step 18: Visualize the ReAct Agent

### Your Task:
Visualize the ReAct agent structure to understand its internal workflow.

**Expected Output:** A diagram showing the ReAct agent's internal structure

In [ ]:
# TODO: Visualize the ReAct agent structure

## Step 19: Stream Responses for Better UX

**Streaming Responses** provide a better user experience by showing incremental progress.

### Your Task:
Implement streaming to see the agent's responses as they're generated.

**Steps:**
1. Use `agent_executor.stream()` instead of `invoke()`
2. Iterate through the streaming events
3. Print each step as it happens

### Benefits of Streaming:
- **Real-time Feedback**: Users see progress immediately
- **Better UX**: No waiting for complete responses
- **Debugging**: Can see each step of the process
- **Transparency**: Users understand what the system is doing

**Expected Output:** Step-by-step streaming of the agent's reasoning and responses

In [ ]:
# TODO: Implement streaming responses
input_message = "Explain the concept of autonomous agents and their capabilities"

# TODO: Stream the response and print each event

## 🎉 Congratulations!

You've successfully built two different types of RAG systems using LangGraph:

### What You've Accomplished:
1. ✅ **Custom LangGraph RAG**: Built a sophisticated workflow with explicit state management
2. ✅ **ReAct Agent**: Used pre-built patterns for rapid development
3. ✅ **Memory Integration**: Added conversation persistence across multiple turns
4. ✅ **Tool Integration**: Created and used custom retrieval tools
5. ✅ **Graph Visualization**: Generated visual representations of workflows
6. ✅ **Streaming Responses**: Implemented real-time response streaming

### Key Takeaways:
- **LangGraph Power**: Enables stateful, multi-step AI applications
- **Flexibility**: Choice between custom graphs and pre-built agents
- **Memory Management**: Crucial for conversational AI systems
- **Tool Integration**: Seamless connection of LLMs with external data sources

### Next Steps:
- Experiment with different document sources
- Add more sophisticated tools
- Implement error handling and validation
- Deploy your system to production

Great work on mastering advanced RAG with LangGraph! 🚀